In [ ]:
import nemo_run as run
from nemo.collections import llm

In [ ]:
# Set path(s) if different:
ROOT_DIR = "/workspace"
MODEL_PATH = f"{ROOT_DIR}/Llama-3.1-8B-nemo"
SEQ_LENGTH = 8192
EXP_NAME = "Llama-3.1-8B-nemo-ft"
EXP_DIR = f"{ROOT_DIR}/{EXP_NAME}"
DATA_PATH = f"{ROOT_DIR}/SAT_DATA-data"
DATA_PATHS = {
    "train": [1.0, f"{DATA_PATH}/SAT_DATA_tokenized_train_text_document"],
    "validation": [f"{DATA_PATH}/SAT_DATA_tokenized_test_text_document"],
    "test": [f"{DATA_PATH}/SAT_DATA_tokenized_val_text_document"],
}
INDEX_MAPPING_DIR = f"{DATA_PATH}/index_mappings"

# Change these to accommodate resources:
DEVICES = 8
NODES = 1
TENSOR_PARALLEL_SIZE = DEVICES
PIPELINE_PARALLEL_SIZE = NODES
MICRO_BATCH_SIZE = 4

# Change the fine-tuning recipe for your model and dataset (below values for demonstration purposes):
STEPS = 30
GLOBAL_BATCH_SIZE = 128
LR = 1e-4
MIN_LR = 1e-5
WARMUP_STEPS = 2
LOG_INTERVAL = 1
VAL_INTERVAL = 10
NUM_VAL_BATCHES = 5


def configure_recipe():
    # Define the recipe
    recipe = llm.llama31_8b.finetune_recipe(
        num_nodes=NODES,
        num_gpus_per_node=DEVICES,
        peft_scheme=None,  # Full finetuning
        seq_length=SEQ_LENGTH,
    )
    recipe.resume.restore_config.path = MODEL_PATH
    recipe.log.explicit_log_dir = EXP_DIR
    recipe.log.ckpt.every_n_train_steps = VAL_INTERVAL

    # Change dataset (default is Squad dataset)
    recipe.data = run.Config(
        llm.PreTrainingDataModule,
        paths=DATA_PATHS,
        index_mapping_dir=INDEX_MAPPING_DIR,
        seq_length=SEQ_LENGTH,
        micro_batch_size=MICRO_BATCH_SIZE,
        global_batch_size=GLOBAL_BATCH_SIZE,
    )

    # Set the training parameters if you dont want to use the recipe defaults
    recipe.trainer.max_steps = STEPS
    recipe.trainer.log_every_n_steps = LOG_INTERVAL
    recipe.trainer.val_check_interval = VAL_INTERVAL
    recipe.trainer.limit_val_batches = NUM_VAL_BATCHES
    recipe.trainer.strategy.tensor_model_parallel_size = TENSOR_PARALLEL_SIZE
    recipe.trainer.strategy.pipeline_model_parallel_size = PIPELINE_PARALLEL_SIZE
    recipe.trainer.strategy.sequence_parallel = TENSOR_PARALLEL_SIZE > 1
    recipe.optim.config.lr = LR
    recipe.optim.lr_scheduler.warmup_steps = WARMUP_STEPS
    recipe.optim.lr_scheduler.min_lr = MIN_LR

    return recipe


recipe = configure_recipe()
print(recipe)
env_vars = {
    "TORCH_NCCL_AVOID_RECORD_STREAMS": "1",  # Disable caching NCCL communication buffer memory
    "NCCL_NVLS_ENABLE": "0",  # Disable NVLink SHARP to save memory
}
executor = run.LocalExecutor(ntasks_per_node=recipe.trainer.devices, launcher="torchrun", env_vars=env_vars)

In [ ]:
run.run(recipe, executor=executor, name=EXP_NAME)

In [ ]:
# PATH To be CHANGED
!mv "{ROOT_DIR}/Llama-3.1-8B-nemo-ft/checkpoints/model_name=0--val_loss=2.03-step=29-consumed_samples=3840.0" "{ROOT_DIR}/Llama-3.1-8B-nemo-ft/checkpoints/best"